In [130]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split, cross_validate
from collections import defaultdict
from datetime import datetime

# Function to get top-N recommendations for each user
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

# Function to calculate precision, recall, and F1-score at k
def precision_recall_f1_at_k(top_n, truth, k=10, threshold=3.5):
    precisions = dict()
    recalls = dict()
    f1_scores = dict()

    for uid, user_ratings in top_n.items():
        relevant_items = set(iid for (iid, actual_rating) in truth[uid] if actual_rating >= threshold)
        predicted_items = set(iid for (iid, _) in user_ratings[:k])
        n_rel = len(relevant_items)
        n_rec_k = len(predicted_items)
        n_rel_and_rec_k = len(relevant_items & predicted_items)

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
        f1_scores[uid] = 2 * (precisions[uid] * recalls[uid]) / (precisions[uid] + recalls[uid]) if (precisions[uid] + recalls[uid]) != 0 else 0

    return precisions, recalls, f1_scores

# Load datasets
movies_df = pd.read_csv('../data/movies.csv')
ratings_df = pd.read_csv('../data/ratings.csv')

# Extract year from the movie titles and create a new column
movies_df['year'] = movies_df['title'].str.extract(r'\((\d{4})\)')

# One-hot encoding of genres
genres_dummies = movies_df['genres'].str.get_dummies(sep='|')
movies_df = pd.concat([movies_df, genres_dummies, movies_df['year']], axis=1)

# Convert timestamp to datetime in ratings_df
ratings_df['date'] = pd.to_datetime(ratings_df['timestamp'], unit='s')
ratings_df['rating_year'] = ratings_df['date'].dt.year
ratings_df['rating_month'] = ratings_df['date'].dt.month

# User-based features
user_stats = ratings_df.groupby('userId')['rating'].agg(['mean', 'std', 'count'])
user_stats.columns = ['user_avg_rating', 'user_rating_std', 'user_rating_count']

# Item-based features
item_stats = ratings_df.groupby('movieId')['rating'].agg(['mean', 'count'])
item_stats.columns = ['movie_avg_rating', 'movie_rating_count']

# Merge features with the main ratings DataFrame
augmented_ratings_df = pd.merge(ratings_df, movies_df, on='movieId')
augmented_ratings_df = pd.merge(augmented_ratings_df, user_stats, on='userId')
augmented_ratings_df = pd.merge(augmented_ratings_df, item_stats, on='movieId')

# Prepare data for Surprise
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(augmented_ratings_df[['userId', 'movieId', 'rating']], reader)

# Train-test split
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Model initialization and training
algo = SVD(random_state=42)
algo.fit(trainset)
predictions = algo.test(testset)

# Calculate and print RMSE and MAE
print('RMSE:', accuracy.rmse(predictions))
print('MAE:', accuracy.mae(predictions))

# Get top-N recommendations
top_n = get_top_n(predictions, n=10)

# Prepare true ratings dictionary for precision, recall, and F1-score calculation
truth = defaultdict(list)
for uid, iid, true_r in augmented_ratings_df[['userId', 'movieId', 'rating']].itertuples(index=False):
    truth[uid].append((iid, true_r))

# Calculate precision, recall, and F1-score
precisions, recalls, f1_scores = precision_recall_f1_at_k(top_n, truth, k=10, threshold=3.5)
avg_precision = sum(prec for prec in precisions.values()) / len(precisions)
avg_recall = sum(rec for rec in recalls.values()) / len(recalls)
avg_f1 = sum(f1 for f1 in f1_scores.values()) / len(f1_scores)

print(f'Average Precision: {avg_precision}')
print(f'Average Recall: {avg_recall}')
print(f'Average F1 Score: {avg_f1}')


RMSE: 0.8715
RMSE: 0.8714954893904284
MAE:  0.6699
MAE: 0.6699008265586093
Average Precision: 0.7371083788706736
Average Recall: 0.15564146206058324
Average F1 Score: 0.23239091424514297


Best precision and recall so far:

- RMSE: 0.8715
- RMSE: 0.8714954893904284
- MAE:  0.6699
- MAE: 0.6699008265586093
- Average Precision: 0.7371083788706736
- Average Recall: 0.15564146206058324
- Average F1 Score: 0.23239091424514297

In [132]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
from collections import defaultdict
from datetime import datetime

def get_top_n(predictions, n=10):
    """
    Returns the top N recommended items for each user, sorted by the estimated rating.

    Args:
        predictions (list of Prediction objects): The list of predictions, as returned by the test method of an algorithm.
        n (int): The number of recommendations to output for each user. Default is 10.

    Returns:
        top_n (dict): A dictionary where keys are user (raw) ids and values are lists of tuples:
                      [(item id, rating estimation), ...] of size n.
    """
    # Initialize the dictionary to hold the top-n items for each user
    top_n = defaultdict(list)
    # Iterate over all predictions
    for uid, iid, true_r, est, _ in predictions:
        # Append the item and its estimated rating to the user's list
        top_n[uid].append((iid, est))
    # Sort the predictions for each user and retrieve the k highest ones
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

def precision_recall_f1_at_k(top_n, truth, k=10, threshold=3.5):
    """
    Calculate precision, recall, and F1-score of the recommendation system based on the top-N recommendations for each user.
    
    Args:
        top_n (dict): A dictionary where the keys are user ids and the values are lists of recommended items (item ids).
        truth (dict): A dictionary where the keys are user ids and the values are lists of tuples (item id, actual rating).
        k (int): The number of top recommendations considered to calculate the metrics (default is 10).
        threshold (float): The rating threshold to consider an item as relevant (default is 3.5).
    
    Returns:
        precisions (dict), recalls (dict), f1_scores (dict): Dictionaries mapping user id to precision, recall, and F1-score.
    """
    # Initialize dictionaries to hold precision, recall, and F1-score
    precisions = dict()
    recalls = dict()
    f1_scores = dict()

    # Iterate over each user's top-N recommendations
    for uid, user_ratings in top_n.items():
        # Find relevant items by checking if their true rating meets the threshold
        relevant_items = set(iid for (iid, actual_rating) in truth[uid] if actual_rating >= threshold)
        # Find predicted items that are in the top-N list
        predicted_items = set(iid for (iid, _) in user_ratings[:k])
        # Count relevant items
        n_rel = len(relevant_items)
        # Count recommended items
        n_rec_k = len(predicted_items)
        # Count relevant and recommended items
        n_rel_and_rec_k = len(relevant_items & predicted_items)

        # Calculate precision
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        # Calculate recall
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
        # Calculate F1-score
        f1_scores[uid] = 2 * (precisions[uid] * recalls[uid]) / (precisions[uid] + recalls[uid]) if (precisions[uid] + recalls[uid]) != 0 else 0

    return precisions, recalls, f1_scores

# Load movie data
movies_df = pd.read_csv('../data/movies.csv')
# Load ratings data
ratings_df = pd.read_csv('../data/ratings.csv')

# Extract year from movie titles and create a new column
movies_df['year'] = movies_df['title'].str.extract(r'\((\d{4})\)')

# One-hot encode genres and merge them with the movie DataFrame
genres_dummies = movies_df['genres'].str.get_dummies(sep='|')
movies_df = pd.concat([movies_df, genres_dummies, movies_df['year']], axis=1)

# Convert UNIX timestamp to datetime in ratings DataFrame
ratings_df['date'] = pd.to_datetime(ratings_df['timestamp'], unit='s')
ratings_df['rating_year'] = ratings_df['date'].dt.year
ratings_df['rating_month'] = ratings_df['date'].dt.month

# Aggregate user statistics from ratings data
user_stats = ratings_df.groupby('userId')['rating'].agg(['mean', 'std', 'count'])
user_stats.columns = ['user_avg_rating', 'user_rating_std', 'user_rating_count']

# Aggregate movie statistics from ratings data
item_stats = ratings_df.groupby('movieId')['rating'].agg(['mean', 'count'])
item_stats.columns = ['movie_avg_rating', 'movie_rating_count']

# Merge user and item statistics back into the ratings DataFrame
augmented_ratings_df = pd.merge(ratings_df, movies_df, on='movieId')
augmented_ratings_df = pd.merge(augmented_ratings_df, user_stats, on='userId')
augmented_ratings_df = pd.merge(augmented_ratings_df, item_stats, on='movieId')

# Prepare the data for the Surprise library
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(augmented_ratings_df[['userId', 'movieId', 'rating']], reader)

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Initialize and train the SVD algorithm
algo = SVD(random_state=42)
algo.fit(trainset)
predictions = algo.test(testset)

# Evaluate the algorithm with RMSE and MAE
print('RMSE:', accuracy.rmse(predictions))
print('MAE:', accuracy.mae(predictions))

# Generate and evaluate top-N recommendations
top_n = get_top_n(predictions, n=10)
truth = defaultdict(list)
for uid, iid, true_r in augmented_ratings_df[['userId', 'movieId', 'rating']].itertuples(index=False):
    truth[uid].append((iid, true_r))

# Calculate and display precision, recall, and F1-score
precisions, recalls, f1_scores = precision_recall_f1_at_k(top_n, truth, k=10, threshold=3.5)
avg_precision = sum(prec for prec in precisions.values()) / len(precisions)
avg_recall = sum(rec for rec in recalls.values()) / len(recalls)
avg_f1 = sum(f1 for f1 in f1_scores.values()) / len(f1_scores)

print(f'Average Precision: {avg_precision}')
print(f'Average Recall: {avg_recall}')
print(f'Average F1 Score: {avg_f1}')


RMSE: 0.8715
RMSE: 0.8714954893904284
MAE:  0.6699
MAE: 0.6699008265586093
Average Precision: 0.7371083788706736
Average Recall: 0.15564146206058324
Average F1 Score: 0.23239091424514297


In [147]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
from collections import defaultdict

# Define the function to extract top-N recommendations
def get_top_n(predictions, n=10):
    """
    Retrieve top-N recommended items for each user based on estimated ratings.
    
    Args:
        predictions (list): List of prediction objects from Surprise.
        n (int): Number of recommendations to retrieve per user.
    
    Returns:
        dict: A dictionary where keys are user IDs and values are lists of tuples (item ID, estimated rating).
    """
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

# Define the function to calculate precision, recall, and F1-score
def precision_recall_f1_at_k(top_n, truth, k=10, threshold=3.5):
    """
    Calculate precision, recall, and F1-score at k metric for each user.
    
    Args:
        top_n (dict): Top-N recommendations per user.
        truth (dict): Actual ratings per user.
        k (int): Number of top recommendations considered for metrics.
        threshold (float): Threshold rating for considering an item as relevant.
    
    Returns:
        dict, dict, dict: Dictionaries containing precision, recall, and F1 scores per user.
    """
    precisions = dict()
    recalls = dict()
    f1_scores = dict()
    
    for uid, user_ratings in top_n.items():
        relevant_items = set(iid for (iid, actual_rating) in truth[uid] if actual_rating >= threshold)
        predicted_items = set(iid for (iid, _) in user_ratings[:k])
        n_rel = len(relevant_items)
        n_rec_k = len(predicted_items)
        n_rel_and_rec_k = len(relevant_items & predicted_items)
        
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
        f1_scores[uid] = 2 * (precisions[uid] * recalls[uid]) / (precisions[uid] + recalls[uid]) if (precisions[uid] + recalls[uid]) != 0 else 0

    return precisions, recalls, f1_scores

# Load datasets
movies_df = pd.read_csv('../data/movies.csv')
ratings_df = pd.read_csv('../data/ratings.csv')

# Extract year from the movie titles and add it as a new column
movies_df['year'] = movies_df['title'].str.extract(r'\((\d{4})\)')

# One-hot encoding of genres
genres_dummies = movies_df['genres'].str.get_dummies(sep='|')
movies_df = pd.concat([movies_df, genres_dummies, movies_df['year']], axis=1)

# Convert timestamp to a datetime object
ratings_df['date'] = pd.to_datetime(ratings_df['timestamp'], unit='s')
# Extract various datetime features
ratings_df['rating_year'] = ratings_df['date'].dt.year
ratings_df['rating_month'] = ratings_df['date'].dt.month
ratings_df['rating_day'] = ratings_df['date'].dt.day
ratings_df['rating_weekday'] = ratings_df['date'].dt.weekday

# Calculate days since the last rating to introduce a 'recency' feature
max_date = ratings_df['date'].max()
ratings_df['days_since_rating'] = (max_date - ratings_df['date']).dt.days

# Calculate user statistics: average, standard deviation, and count of ratings
user_stats = ratings_df.groupby('userId')['rating'].agg(['mean', 'std', 'count'])
user_stats.columns = ['user_avg_rating', 'user_rating_std', 'user_rating_count']

# Calculate movie statistics: average rating and count of ratings
item_stats = ratings_df.groupby('movieId')['rating'].agg(['mean', 'count'])
item_stats.columns = ['movie_avg_rating', 'movie_rating_count']

# Merge all statistics and features into the main ratings DataFrame
augmented_ratings_df = pd.merge(ratings_df, movies_df, on='movieId')
augmented_ratings_df = pd.merge(augmented_ratings_df, user_stats, on='userId')
augmented_ratings_df = pd.merge(augmented_ratings_df, item_stats, on='movieId')

# Initialize the Surprise Reader and Dataset
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(augmented_ratings_df[['userId', 'movieId', 'rating']], reader)

# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Train an SVD model on the training set
algo = SVD(random_state=42)
algo.fit(trainset)
predictions = algo.test(testset)

# Calculate and print RMSE and MAE
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)
print(f'RMSE: {rmse}, MAE: {mae}')

# Generate top-N recommendations
top_n = get_top_n(predictions, n=10)

# Prepare true ratings data for evaluation
truth = defaultdict(list)
for uid, iid, true_r in augmented_ratings_df[['userId', 'movieId', 'rating']].itertuples(index=False):
    truth[uid].append((iid, true_r))

# Calculate and print precision, recall, and F1-score
precisions, recalls, f1_scores = precision_recall_f1_at_k(top_n, truth, k=10, threshold=3.5)
avg_precision = sum(prec for prec in precisions.values()) / len(precisions)
avg_recall = sum(rec for rec in recalls.values()) / len(recalls)
avg_f1 = sum(f1 for f1 in f1_scores.values()) / len(f1_scores)
print(f'Average Precision: {avg_precision}, Average Recall: {avg_recall}, Average F1 Score: {avg_f1}')


RMSE: 0.8715
MAE:  0.6699
RMSE: 0.8714954893904284, MAE: 0.6699008265586093
Average Precision: 0.7371083788706736, Average Recall: 0.15564146206058324, Average F1 Score: 0.23239091424514297


In [ ]:
RMSE: 0.8715
RMSE: 0.8714954893904284
MAE:  0.6699
MAE: 0.6699008265586093
Average Precision: 0.7371083788706736
Average Recall: 0.15564146206058324
Average F1 Score: 0.23239091424514297

In [149]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
from collections import defaultdict

# Assuming your datasets are loaded as follows:
movies_df = pd.read_csv('../data/movies.csv')
ratings_df = pd.read_csv('../data/ratings.csv')

# Step 1: Define a new user profile
# Create a list of movies this user likes
# Example: User likes mostly Sci-Fi and Action
user_preferences = [
    {'movieId': 260, 'rating': 4.5},  # Star Wars IV
    {'movieId': 1196, 'rating': 5}, # Star Wars V
    {'movieId': 1210, 'rating': 4.5}, # Star Wars VI
    {'movieId': 2628, 'rating': 4}, # Star Wars I
    {'movieId': 541, 'rating': 5},  # Blade Runner
    {'movieId': 1265, 'rating': 3}, # Groundhog Day
    {'movieId': 2571, 'rating': 5}  # Matrix
]

# Assign a new unique user ID
new_user_id = ratings_df['userId'].max() + 1

# Create a DataFrame for the new user
new_user_ratings = pd.DataFrame(user_preferences)
new_user_ratings['userId'] = new_user_id

# Step 2: Add new user ratings to the existing DataFrame
ratings_df = pd.concat([ratings_df, new_user_ratings], ignore_index=True)

# Step 3: Prepare the data and train the model
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Step 4: Generate recommendations for the new user
# Get a list of all movie ids
all_movies = movies_df['movieId'].unique()
# Predict ratings for all movies that the user hasn't rated
testset = [[new_user_id, movie_id, 4.] for movie_id in all_movies if movie_id not in new_user_ratings['movieId'].values]
predictions = algo.test(testset)

# Extract the top 10 recommendations
top_n = get_top_n(predictions, n=10)

# Print the recommended movie titles
recommended_movie_ids = [iid for (iid, _) in top_n[new_user_id]]
recommended_movies = movies_df[movies_df['movieId'].isin(recommended_movie_ids)]
print(recommended_movies[['title', 'genres']])


                                                  title  \
277                    Shawshank Redemption, The (1994)   
602   Dr. Strangelove or: How I Learned to Stop Worr...   
906                           Lawrence of Arabia (1962)   
971                                       Patton (1970)   
2226                                  Fight Club (1999)   
2623                     Outlaw Josey Wales, The (1976)   
4176                City of God (Cidade de Deus) (2002)   
4909       Eternal Sunshine of the Spotless Mind (2004)   
6315                               Departed, The (2006)   
6710                            Dark Knight, The (2008)   

                                       genres  
277                               Crime|Drama  
602                                Comedy|War  
906                       Adventure|Drama|War  
971                                 Drama|War  
2226              Action|Crime|Drama|Thriller  
2623  Action|Adventure|Drama|Thriller|Western  
4176    Action

In [151]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split, PredefinedKFold
from collections import defaultdict

# Load movie data
movies_df = pd.read_csv('../data/movies.csv')
ratings_df = pd.read_csv('../data/ratings.csv')

# Function to search movies by title
def search_movies(movies_df, search_query):
    return movies_df[movies_df['title'].str.contains(search_query, case=False, na=False)]

# Function to get movie recommendations for a user
def get_recommendations(user_ratings, movies_df, ratings_df):
    # Add a new user ID to the ratings
    new_user_id = ratings_df['userId'].max() + 1
    user_data = pd.DataFrame(user_ratings, columns=['movieId', 'rating'])
    user_data['userId'] = new_user_id
    
    # Append the new user data to the original ratings DataFrame
    augmented_ratings_df = pd.concat([ratings_df, user_data])
    
    # Train the SVD model
    reader = Reader(rating_scale=(0.5, 5))
    data = Dataset.load_from_df(augmented_ratings_df[['userId', 'movieId', 'rating']], reader)
    trainset = data.build_full_trainset()
    algo = SVD()
    algo.fit(trainset)
    
    # Predict ratings for all movies that the user hasn't rated
    testset = [[new_user_id, mid, 4.] for mid in movies_df['movieId'].unique() if mid not in user_data['movieId'].tolist()]
    predictions = algo.test(testset)
    
    # Get the top 10 movie recommendations
    top_n = get_top_n(predictions, n=10)
    recommended_ids = [iid for (iid, _) in top_n[new_user_id]]
    return movies_df[movies_df['movieId'].isin(recommended_ids)]

# Interactive movie rating input
def user_rating_input():
    user_ratings = []
    while True:
        search_query = input("Enter a movie title to search (or type 'done' to finish): ")
        if search_query.lower() == 'done':
            break
        found_movies = search_movies(movies_df, search_query)
        if found_movies.empty:
            print("No movies found, try again!")
            continue
        print(found_movies[['movieId', 'title']])
        
        movie_id = input("Enter movie ID to rate: ")
        rating = float(input("Enter your rating (0.5 to 5.0): "))
        user_ratings.append({'movieId': int(movie_id), 'rating': rating})
    return user_ratings

# Example usage
user_ratings = user_rating_input()
recommended_movies = get_recommendations(user_ratings, movies_df, ratings_df)
print("Recommended movies for you:")
print(recommended_movies[['title', 'genres']])


Enter a movie title to search (or type 'done' to finish):  Star Wars Episode V


No movies found, try again!


Enter a movie title to search (or type 'done' to finish):  Star Wars


      movieId                                              title
224       260          Star Wars: Episode IV - A New Hope (1977)
898      1196  Star Wars: Episode V - The Empire Strikes Back...
911      1210  Star Wars: Episode VI - Return of the Jedi (1983)
1979     2628   Star Wars: Episode I - The Phantom Menace (1999)
3832     5378  Star Wars: Episode II - Attack of the Clones (...
5896    33493  Star Wars: Episode III - Revenge of the Sith (...
6823    61160                   Star Wars: The Clone Wars (2008)
7367    79006  Empire of Dreams: The Story of the 'Star Wars'...
8683   122886  Star Wars: Episode VII - The Force Awakens (2015)
8908   135216               The Star Wars Holiday Special (1978)
9433   166528                Rogue One: A Star Wars Story (2016)
9645   179819                    Star Wars: The Last Jedi (2017)
9710   187595                     Solo: A Star Wars Story (2018)


Enter movie ID to rate:  1196
Enter your rating (0.5 to 5.0):  5
Enter a movie title to search (or type 'done' to finish):  Raiders of the Lost Ark


...

      movieId                                              title
969      1270                          Back to the Future (1985)
1486     2011                  Back to the Future Part II (1989)
1487     2012                 Back to the Future Part III (1990)
8163   102666  Ivan Vasilievich: Back to the Future (Ivan Vas...


Enter movie ID to rate:  1270
Enter your rating (0.5 to 5.0):  5
Enter a movie title to search (or type 'done' to finish):  Blade Runner


      movieId                     title
474       541       Blade Runner (1982)
9604   176371  Blade Runner 2049 (2017)


Enter movie ID to rate:  176371
Enter your rating (0.5 to 5.0):  5
Enter a movie title to search (or type 'done' to finish):  done


Recommended movies for you:
                                                  title  \
224           Star Wars: Episode IV - A New Hope (1977)   
277                    Shawshank Redemption, The (1994)   
602   Dr. Strangelove or: How I Learned to Stop Worr...   
690                           North by Northwest (1959)   
863              Monty Python and the Holy Grail (1975)   
906                           Lawrence of Arabia (1962)   
909                               Apocalypse Now (1979)   
926                                      Amadeus (1984)   
1616                             Rosemary's Baby (1968)   
7644  Harry Potter and the Deathly Hallows: Part 2 (...   

                                           genres  
224                       Action|Adventure|Sci-Fi  
277                                   Crime|Drama  
602                                    Comedy|War  
690     Action|Adventure|Mystery|Romance|Thriller  
863                      Adventure|Comedy|Fantasy  
906       